In [29]:
# Pandas import and reading and preparing the population density csv
import pandas as pd
pd.options.mode.chained_assignment = None
pop_dens = pd.read_csv('population density.csv')
pop_dens['name'][19] = 'Totteridge & Woodside'
pop_dens.set_index('name', inplace=True);

# reading the out_60 csv, containing each individual crime
dataset = pd.read_csv('Out_60.csv')
dataset = dataset.drop(["Unnamed: 0", 'last_outcome_category', 'location'], axis=1)
mask = dataset['ward_name']!=dataset['ward_name']
dataset = dataset[~mask]

In [28]:
# Showing first 5 rows of pop_dens for understanding of the data
pop_dens.head()

,id,boundaries,Population,Area (km2),Population density
name,,,,,
Burnt Oak,E05013630,"[(51.597724278003, -0.25291781885811), (51.597...",16869,2.5460,6625.687353
Barnet Vale,E05013628,"[(51.634276016337, -0.17977180627771), (51.634...",16211,3.5170,4609.326130
Brunswick Park,E05013629,"[(51.627511902482, -0.15625171052302), (51.627...",21857,3.0480,7170.931759
Childs Hill,E05013631,"[(51.559882977663, -0.19691984536718), (51.559...",19629,3.1560,6219.581749
Colindale North,E05013632,"[(51.597724278003, -0.25291781885811), (51.597...",12060,0.9976,12089.013633


In [31]:
# Showing first 5 rows of dataset for understanding of the data
dataset.head()

,crime_id,month,longitude,latitude,lsoa_code,lsoa_name,ward_code,ward_name
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,2010-12,-0.201877,51.655538,E01000248,Barnet 001A,E05013644,High Barnet
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,2010-12,-0.207853,51.654317,E01000248,Barnet 001A,E05013644,High Barnet
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2010-12,-0.202510,51.656348,E01000248,Barnet 001A,E05013644,High Barnet
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,2010-12,-0.206779,51.654768,E01000248,Barnet 001A,E05013644,High Barnet
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,2010-12,-0.209537,51.655223,E01000249,Barnet 001B,E05013644,High Barnet


In [20]:
def create_monthly_crimes(month):
    # Function that can return a dictionary of crimes per ward in a given month
    # Used to test the demand allocation algorithm
    monthly_crimes = {}
    for w in wards:
        try:
            monthly_crimes[w] = int(dataset[dataset['ward_name'] == w].groupby('month').count()['longitude'][month])
        except:
            monthly_crimes[w] = 0
            
    return monthly_crimes

In [ ]:
def fix_amount(rounded_total, temp_dct):
    # Function that fixes the assignment of officers based on certain limitations. 
    # As well as fixing the weird rounding issue that python can have.
    
    # If the rounded total is higher than 50, 
    # finds the wards which have the current highest amount of assigned officers (rounded).
    while rounded_total > 50:
        high = -999
        for x in temp_dct:
            if temp_dct[x][0] > high:
                high = temp_dct[x][0]
                highest = [x]
            elif temp_dct[x][0] == high:
                highest.append(x)
         
        
        # Of the wards with the highest number of assigned officers, finds the single ward with the lowest unrounded
        # number of assigned officers
        low = 999
        for x in highest:
            if temp_dct[x][1] < low:
                low = temp_dct[x][1]
                final = x
    
        # Removes one assigned officer for the found ward, and removes one from rounded_total, 
        # continues untill exactly 50 officers are assigned
        temp_dct[final][1] -= 1
        temp_dct[final][0] -= 1      
        rounded_total -= 1.0
        
        
    # If the rounded total is lower than 50, 
    # finds the wards which have the current lowest amount of assigned officers (rounded).
    while rounded_total < 50:
        low = 999
        for x in temp_dct:
            if temp_dct[x][0] < low:
                low = temp_dct[x][0]
                lowest = [x]
            elif temp_dct[x][0] == low:
                lowest.append(x)

        # Of the wards with the lowest number of assigned officers, finds the single ward with the highest unrounded
        # number of assigned officers
        high = -999
        for x in lowest:
            if temp_dct[x][1] > high:
                high = temp_dct[x][1]
                final = x  

        # Adds one assigned officer to the found ward, and adds one to rounded_total, 
        # continues untill exactly 50 officers are assigned
        temp_dct[final][1] += 1
        temp_dct[final][0] += 1
        rounded_total +=1
        
               
    # Finds all the wards which have 0 officers assigned to them.
    no_officers = []
    for x in temp_dct:
        if temp_dct[x][0] == 0:
            no_officers.append(x)


    # Finds the wards which have the highest number of officers assigned to them unrounded, 
    # adds to list and removes one from that ward, 
    # continue untill the same amount of wards are added as there are wards with no officers
    current_highest = []
    while len(current_highest) != len(no_officers):
        high = -999
        for x in temp_dct:
            if temp_dct[x][1] > high:
                high = temp_dct[x][1]
                highest = x

        current_highest.append(highest)
        temp_dct[highest][1] -= 1

        
    # Adds an officer to the wards with no officers
    for no_of in no_officers:
        temp_dct[no_of][0] += 1
        
    # Removes an officer from the wards assigned to the highest officer list.
    for cur_h in current_highest:
        temp_dct[cur_h][0] -= 1

In [21]:
# Set dictionary of monthly crimes, used for testing the demand allocation algorithm
monthly_crimes = {'High Barnet': 6, 'Underhill': 3, 'East Barnet': 2, 'Barnet Vale': 5, 'Edgwarebury': 5, 'Totteridge & Woodside': 5, 'Whetstone': 6, 'Brunswick Park': 6, 'Woodhouse': 5, 'Edgware': 8, 'Friern Barnet': 7, 'Mill Hill': 8, 'Burnt Oak': 10, 'West Finchley': 11, 'Finchley Church End': 6, 'Colindale North': 3, 'East Finchley': 9, 'Hendon': 2, 'Garden Suburb': 5, 'West Hendon': 9, 'Golders Green': 6, 'Childs Hill': 7, 'Cricklewood': 3,'Colindale South': 2}

In [22]:
# Adds the crimes per ward of the monthly_crimes dictionary to the pop_dens datafrmae
crimes = [monthly_crimes[x] for x in pop_dens.index]
pop_dens["crimes"] = crimes

# Adds the amount of crimes relative to the square kilometer area of the ward to the pop_dens dataframe
crimes_area = [(pop_dens['crimes'][x]/pop_dens['Area (km2)'][x]) for x in pop_dens.index]
pop_dens['crimes_area'] = crimes_area

# Calculates how many crimes happen in a ward relative to the total of all wards based on the crimes per area
# adds this value per ward to the pop_dens dataframe
total_crimes_area = sum(crimes_area)
relative_crimes_area = [(x/total_crimes_area) for x in crimes_area]
pop_dens['relative_crimes_area'] = relative_crimes_area

In [23]:
# Shows the newly added columns
pop_dens.head()

,id,boundaries,Population,Area (km2),Population density,crimes,crimes_area,relative_crimes_area
name,,,,,,,,
Burnt Oak,E05013630,"[(51.597724278003, -0.25291781885811), (51.597...",16869,2.5460,6625.687353,10,3.927730,0.084675
Barnet Vale,E05013628,"[(51.634276016337, -0.17977180627771), (51.634...",16211,3.5170,4609.326130,5,1.421666,0.030649
Brunswick Park,E05013629,"[(51.627511902482, -0.15625171052302), (51.627...",21857,3.0480,7170.931759,6,1.968504,0.042438
Childs Hill,E05013631,"[(51.559882977663, -0.19691984536718), (51.559...",19629,3.1560,6219.581749,7,2.217997,0.047816
Colindale North,E05013632,"[(51.597724278003, -0.25291781885811), (51.597...",12060,0.9976,12089.013633,3,3.007217,0.064831


In [24]:
# Creates a temporary dictionary of the wards with a list of their relative crimes per area * 50, both rounded and unrounded
temp_dct = {x: [round(pop_dens['relative_crimes_area'][x]*50), pop_dens['relative_crimes_area'][x]*50] for x in pop_dens.index}

# Calculates the rounded_total as a parameter for the fix_amount function
rounded_total = sum([round(x*50) for x in pop_dens['relative_crimes_area']])

# Runs the fix_amount function on the temp_dct with rounded_total as a parameter
fix_amount(rounded_total, temp_dct)

# Small for loop to take the temp_dct values and use it to assign officers per ward (*2 because assignment in pairs)
assigned_officers = {}
for x in temp_dct:
    assigned_officers[x] = temp_dct[x][0]*2

# Calculates the final total amount of assigned officers to check for correctness
final_total = 0
for x in assigned_officers:
    final_total += assigned_officers[x]
    
# Prints the total assigned officers, and where each officers has been assigned.
print("Total assigned officers this month:", final_total)
assigned_officers

Total assigned officers this month: 100


{'Burnt Oak': 8,
 'Barnet Vale': 4,
 'Brunswick Park': 4,
 'Childs Hill': 4,
 'Colindale North': 6,
 'Colindale South': 2,
 'Cricklewood': 4,
 'East Barnet': 2,
 'East Finchley': 6,
 'Edgware': 4,
 'Edgwarebury': 2,
 'Finchley Church End': 4,
 'Friern Barnet': 6,
 'Garden Suburb': 2,
 'Golders Green': 8,
 'Hendon': 2,
 'West Hendon': 2,
 'High Barnet': 2,
 'Mill Hill': 2,
 'Totteridge & Woodside': 6,
 'Underhill': 2,
 'West Finchley': 6,
 'Whetstone': 6,
 'Woodhouse': 6}